In [3]:
"""Word Count Engine
===============================
Implement a document scanning function wordCountEngine, which receives a string 
document and returns a list of all unique words in it and their number of
occurrences, sorted by the number of occurrences in a descending order. If
two or more words have the same count, they should be sorted according to
their order in the original sentence. Assume that all letters are in english
alphabet. You function should be case-insensitive, so for instance, the words
“Perfect” and “perfect” should be considered the same word.

The engine should strip out punctuation (even in the middle of a word) and
use whitespaces to separate words.

Analyze the time and space complexities of your solution. Try to optimize
for time while keeping a polynomial space complexity.

Examples:

input:  document = "Practice makes perfect. you'll only
                    get Perfect by practice. just practice!"

output: [ ["practice", "3"], ["perfect", "2"],
          ["makes", "1"], ["youll", "1"], ["only", "1"], 
          ["get", "1"], ["by", "1"], ["just", "1"] ]
"""

"""My solution :  Wrong output, un-optmized"""
from collections import defaultdict
import string

def word_count_engine(document):
    docList = document.lower().split(' ')
    wordMap = defaultdict(list)
    try:
        for wordIndex in range(0, len(docList)):
            word = docList[wordIndex]
            count = 0
            order = 0
            for letter in word:
                if letter not in string.ascii_letters:
                    word = word.replace(letter, "")
            if word in wordMap:
                count = wordMap[word][0] + 1
                order = wordMap[word][1]
            else:
                count = 1
                order = wordIndex
                
            if word == "":
                pass
            else:
                wordMap[word] = [count, order]
    except Exception as e:
        print(e)
    
    wordList = sorted(wordMap.values(), key=lambda x: x[0], reverse=True)
    output = []
    for value in wordList:
        for k, v in wordMap.items():
            if value == v:
                output.append([k,str(v[0])])
    
    return output
    
"""Correct solution :  optmized (time O(n), space O(n))
========================================================
Let document consist of of N words where M of them are unique (M ≤ N). 
The solution consists of two steps: 
1) parsing the string according to the criteria described in the problem 
and counting the number of occurrences of each word. 
2) sorting the [word, occurrence] pairs by the number of words’ occurrences 
in a descending order.

Step 1: we tokenize document into words by using whitespaces as delimiters.
For each word, we clean it from all non-alphabetic characters (digits, punctuation etc)
and convert it to lowercase to make counting case-insensitive. In this part, you should 
be leveraging whatever parsing capabilities your programming language of choice is 
providing. There is really no point of implementing functions that already exist.
As for counting, we’ll use a Map (Hash Table) to store words and their corresponding 
occurrences. A map is optimal in this case because it allows us find, store and
update operations in O(1) time complexity.

Step 2: as for the sorting part, rather than sorting the entries in the map
directly, which takes O(M⋅log(M)) - where M is number of unique words in 
document - a better solution will be to place words into an array of string
arrays indexed by the occurrence number and then iterate through the array 
in the reverse order. This is similar to a Bucket Sort. 
The proposed solution trades off a bit of space for performance, 
which may be a reasonable trade under certain circumstances.

Time Complexity: let N be the number of words in document and M 
the number of unique words in it (M ≤ N). Iterating over all words,
cleaning them and inserting them into a map takes O(N). 
The sorting step takes O(M) since notice that in the second loop, 
every word gets visited only once. The total time complexity is 
therefore O(N + M), which is O(N).

Space Complexity: wordMap takes O(M) space and the array of
strings array, counterList, takes another O(M). So, in total,
the space complexity is O(M).

Note: the reason we’re analyzing the problem complexity 
in terms of the number of words, and not number of characters 
is because the average length of an english word is ~5, 
so from a practical perspective this could be regarded as a
constant and therefore can be ignored (i.e. O(5N) = O(N))
"""
def wordCountEngineOPTIMIZED(document):
    wordMap = {}
    wordList = document.lower().split(' ')
    
    for index in range(0, len(wordList)):
        currentWord = wordList[index]
        currentWordCount = 0
        
        for letter in currentWord: 
            if (ord(letter) < ord('a') or ord(letter) > ord('z')):
                # alternatively, you can also do ==> 
                #import string
                #if letter not in string.ascii_letters:
                currentWord = currentWord.replace(letter, "")
                            
        if currentWord in wordMap:
            currentWordCount = wordMap[currentWord] + 1
        else:
            currentWordCount = 1
                            
        if currentWord == "":
            pass
        else:
            wordMap[currentWord] = currentWordCount
                            
    counterList = [None for i in range(0, len(wordMap))]
    
    # add all words to a list indexed by the
    # corresponding occurrence number.
    for word, count in wordMap.items():
        wordCounterList = counterList[count]
        if (wordCounterList == None):
            wordCounterList = []
        wordCounterList.append(word)
        counterList[count] = wordCounterList

    # iterate through the list in reverse order (largest count to lowest count)
    # and add only non-null values to result
    result = []
    for count in range(len(counterList)-1, -1, -1):
        listOfWordsWithSameCount = counterList[count]
        if (listOfWordsWithSameCount == None or len(listOfWordsWithSameCount) == 0):
            continue
        else:
            for word in listOfWordsWithSameCount:
                result.append([word, str(count)])

    return result

document1 = "Practice makes perfect. you'll only get Perfect makes by practice. just practice!"
expected1 = "[['practice', '3'], ['makes', '2'], ['perfect', '2'], ['youll', '1'], ['only', '1'], ['get', '1'], ['by', '1'], ['just', '1']]" 
print("input: ", expected1)
#print("\nMy solution: {}".format(word_count_engine(document1)))
print("\nCorrect solution: {}".format(wordCountEngineOPTIMIZED(document1)))
print(expected1.strip() == str(word_count_engine(document1)).strip() == str(wordCountEngineOPTIMIZED(document1)).strip())

input:  [['practice', '3'], ['makes', '2'], ['perfect', '2'], ['youll', '1'], ['only', '1'], ['get', '1'], ['by', '1'], ['just', '1']]

Correct solution: [['practice', '3'], ['makes', '2'], ['perfect', '2'], ['youll', '1'], ['only', '1'], ['get', '1'], ['by', '1'], ['just', '1']]
True


In [ ]:
"""      
Flatten a Dictionary
input:  dict = {
            "Key1" : "1",
            "Key2" : {
                "a" : "2",
                "b" : "3",
                "c" : {
                    "d" : "3",
                    "e" : {
                        "" : "1"
                    }
                }
            }
        }

output: {
            "Key1" : "1",
            "Key2.a" : "2",
            "Key2.b" : "3",
            "Key2.c.d" : "3",
            "Key2.c.e" : "1"
        }
A recursion is natural choice for this kind of problem. 
We iterate over the keys in dict and distinguish between two cases:
If the value mapped to a key is a primitive,
we take that key and simply concatenate it to the flattened 
key we created up to this point. We then map the resultant key 
to the value in the output dictionary. If the value is a dictionary,
we do the same concatenation, but instead of mapping the result
to the value in the output dictionary,
we recurse on the value with the newly formed key.

Time Complexity: O(N), where N is the number of keys in the input dictionary. 
We visit every key in dictionary only once, hence the linear time complexity.

Space Complexity: O(N) since the output dictionary is asymptotically as big as 
the input dictionary. We also store recursive calls in the execution 
stack which in the worst case scenario could be O(N), as well. 
The total is still O(N).
"""

def flattenDictionary(dictionary):
    flatDictionary = {}
    flattenDictionaryHelper("", dictionary, flatDictionary)
    
    return flatDictionary

def removeLastDot(key):
    if key.endswith("."):
        return key[:-1]
    else:
        return key

def flattenDictionaryHelper(initialKey, dictionary, flatDictionary):
    for key, value in dictionary.items():
        if (type(value) != dict): # the value is of a primitive type
            if ((initialKey == None) | (initialKey == "")):
                flatDictionary[key] = value
            else:
                newKey = removeLastDot(initialKey + "." + key)
                flatDictionary[newKey] = value
        else:
            if ((initialKey == None) | (initialKey == "")):
                flattenDictionaryHelper(key, value, flatDictionary)
            else:
                newKey = removeLastDot(initialKey + "." + key)
                flattenDictionaryHelper(newKey, value, flatDictionary)

d = { "Key1" : "1",
      "Key2" : {
        "a" : "2",
        "b" : 3,
        "c" : {
            "d" : "4",
            "e" : {
                "" : 5,
                'k': 6
            }
        }
    }
}
print(flattenDictionary(d))

In [4]:
"""
An H-tree is a geometric shape that consists of a repeating pattern
resembles the letter “H”.

It can be constructed by starting with a line segment 
of arbitrary length, drawing two segments of the same length
at right angles to the first through its endpoints, and continuing 
in the same vein, reducing (dividing) the length of the line segments
drawn at each stage by √2.

Write a function drawHTree that constructs an H-tree, 
given its center (x and y coordinates), a starting length,
and depth. Assume that the starting line is parallel to the X-axis.

Use the function drawLine provided to implement your algorithm.
In a production code, a drawLine function would render a real line
between two points. However, this is not a real production environment,
so to make things easier, implement drawLine such that it simply prints
its arguments (the print format is left to your discretion).

Analyze the time and space complexity of your algorithm.
In your analysis, assume that drawLine's time and space complexities
are constant, i.e. O(1).

We will start from the center point. Compute the coordinates of the 4 
tips of the H. Then we shall draw the 3 line segments of the H, i.e. 
left and right vertical of the H, and the connection of the two vertical 
segments. We will update the length and recursively draw 4 half-size 
H-trees of order one less than the current depth.

Time Complexity: every call of drawHTree invokes 
9 expressions whose time complexity is O(1) and 4 calls of drawHTree
until depth(denoted here as D) reaches to 0. 
Therefore: T(D) = 9 + 4 * T(D-1), where T is the time complexity
function and D is the depth of the H-Tree. 
Now, if we expand T(D-1) recursively all the way to T(0),
it’ll be easy to see that T(D) = O(4^D).

Space Complexity:  recursive calls add overhead since we store them 
in the execution stack. The space occupied in the stack will be then O(D),
in the worst case scenario. 
The stack space occupied will be no more than O(D) at any given
point since a sibling drawHTree will not be called before the current
one being executed returns (i.e. finishes its execution).
"""
import math 
import turtle

def drawLine(x1, y1, x2, y2):
    # draws line, assume implementation available
    print("({x1},{y1}) , ({x2},{y2})".format(x1=x1, y1=y1, x2=x2, y2=y2))
#     point1 = (x1, y1)
#     point2 = (x2, y2)
#     turtle.penup()
#     turtle.goto(point1)
#     turtle.pendown()
#     turtle.goto(point2)
#     #turtle.hideturtle()
#     turtle.exitonclick()

def drawHTree(x, y, length, depth):
    # recursion base case
    if (depth == 0):
        return

    x1 = x - length/2
    x2 = x + length/2
    y1 = y - length/2
    y2 = y + length/2

    # draw the 3 line segments of the H-Tree
    drawLine(x1, y1, x1, y2)    # left segment
    drawLine(x2, y1, x2, y2)    # right segment
    drawLine(x1, y,  x2,  y)    # connecting segment

    # at each stage, the length of segments decreases by a factor of √2
    newLength = length/math.sqrt(2)

    # decrement depth by 1 and draw an H-tree
    # at each of the tips of the current ‘H’
    drawHTree(x1, y1, newLength, depth-1)     # lower left  H-tree
    drawHTree(x1, y2, newLength, depth-1)     # upper left  H-tree
    drawHTree(x2, y1, newLength, depth-1)     # lower right H-tree
    drawHTree(x2, y2, newLength, depth-1)     # upper right H-tree

drawHTree(0.0, 0.0, 140, 3)

(-70.0,-70.0) , (-70.0,70.0)
(70.0,-70.0) , (70.0,70.0)
(-70.0,0.0) , (70.0,0.0)
(-119.49747468305833,-119.49747468305833) , (-119.49747468305833,-20.50252531694168)
(-20.50252531694168,-119.49747468305833) , (-20.50252531694168,-20.50252531694168)
(-119.49747468305833,-70.0) , (-20.50252531694168,-70.0)
(-154.49747468305833,-154.49747468305833) , (-154.49747468305833,-84.49747468305833)
(-84.49747468305833,-154.49747468305833) , (-84.49747468305833,-84.49747468305833)
(-154.49747468305833,-119.49747468305833) , (-84.49747468305833,-119.49747468305833)
(-154.49747468305833,-55.50252531694167) , (-154.49747468305833,14.497474683058314)
(-84.49747468305833,-55.50252531694167) , (-84.49747468305833,14.497474683058314)
(-154.49747468305833,-20.50252531694168) , (-84.49747468305833,-20.50252531694168)
(-55.50252531694167,-154.49747468305833) , (-55.50252531694167,-84.49747468305833)
(14.497474683058314,-154.49747468305833) , (14.497474683058314,-84.49747468305833)
(-55.50252531694167,-119.4

In [1]:
"""
Pairs with Specific Difference
Given an array arr of distinct integers and a nonnegative integer k, 
write a function findPairsWithGivenDifference that returns an array of
all pairs [x,y] in arr, such that x - y = k. If no such pairs exist, 
return an empty array.

Note: the order of the pairs in the output array should maintain the
order of the y element in the original array.
Examples:
    input:  arr = [0, -1, -2, 2, 1], k = 1
    output: [[1, 0], [0, -1], [-1, -2], [2, 1]]

    input:  arr = [1, 7, 5, 3, 32, 17, 12], k = 17
    output: []

Constraints:
    [time limit] 5000ms
    [input] array.integer arr
    0 ≤ arr.length ≤ 100
    [input]integer k
    k ≥ 0
    [output] array.array.integer

Pairs with Specific Difference
A naive approach is is to run two loops. 
The outer loop picks the first element (smaller element) 
and the inner loop looks up for the element picked 
by the outer loop plus k. 
While this solution is done in O(1) space complexity, 
its time complexity is O(N^2), which isn’t asymptotically optimal.

We can use a hash map to improve the time complexity to O(N⋅log(N))
for the worst case and O(N) for the average case. 
We rely on the fact that if x - y = k then x - k = y.

The first step is to traverse the array, and for each element arr[i], 
we add a key-value pair of (arr[i] - k, arr[i]) to a hash map. 
Once the map is populated, we traverse the array again, 
and check for each element if a match exists in the map.

Both the first and second steps take O(N⋅log(N)) for the worst
case and O(N) for the average case. So the overall time complexity
is O(N) for the average case.
    
"""
import itertools

# My Solution: Time: O(n^2), Space:  O(n)
def find_pairs_with_given_difference(arr, k):
    if (len(arr) == 0):
        return []
    
    if k < 0:
        return
    
    result_arr = []
    for elem in arr:
        for (x,y) in itertools.product(arr, [elem]):
            if x - y == k:
                #print("({x},{y}) = {k}: TRUE".format(x=x, y=y, k=k))
                result_arr.append([x, y])
            else:
                pass
    return result_arr

# Correct Solution: Time: O(N) , Space O(n)
def findPairsWithGivenDifference(arr, k):
    # since we don't allow duplicates, no pair can satisfy x - 0 = y
    if k == 0:
        return arr
        
    dictionary = {}
    answer = []
    """
    if x - y = k, then
    x - k = y
    """
    for x in arr:
        y = x - k
        dictionary[y] = x
    
    for y in arr:
        if y in dictionary:
            x = dictionary[y]
            answer.append([x, y]) 
            
    return answer

arr = [0, -1, -2, 2, 1]
k = 1
arr2 = [1, 7, 5, 3, 32, 17, 12]
k2 = 17

print("My Brute-force solution: O(n^2)")
print("result1:", find_pairs_with_given_difference(arr, k))
print("result2: ", find_pairs_with_given_difference(arr2, k2))

print("\nFaster solution: O(2n), i.e., O(n)")
print("result1:", findPairsWithGivenDifference(arr, k))
print("result2: ", findPairsWithGivenDifference(arr2, k2))

My Brute-force solution: O(n^2)
result1: [[1, 0], [0, -1], [-1, -2], [2, 1]]
result2:  []

Faster solution: O(2n), i.e., O(n)
result1: [[1, 0], [0, -1], [-1, -2], [2, 1]]
result2:  []


In [ ]:
"""
====================== Award Budget Cuts ============================
The awards committee of your alma mater (i.e. your college/university) 
asked for your assistance with a budget allocation problem they’re facing. 
Originally, the committee planned to give N research grants this year.
However, due to spending cutbacks, the budget was reduced to newBudget 
dollars and now they need to reallocate the grants. The committee made a 
decision that they’d like to impact as few grant recipients as possible by
applying a maximum cap on all grants. Every grant initially planned to be 
higher than cap will now be exactly cap dollars. Grants less or equal to cap, 
obviously, won’t be impacted.

Given an array grantsArray of the original grants and the reduced budget 
newBudget, write a function findGrantsCap that finds in the most efficient
manner a cap such that the least number of recipients is impacted and that
the new budget constraint is met (i.e. sum of the N reallocated grants 
equals to newBudget).

Analyze the time and space complexities of your solution.

Example:
input:  grantsArray = [2, 100, 50, 120, 1000]
        newBudget = 190

output: 47 # and given this cap the new grants array would be
           # [2, 47, 47, 47, 47]. Notice that the sum of the
           # new grants is indeed 190
           
Constraints:
    [time limit] 5000ms
    [input] array.double grantsArray
    0 ≤ grantsArray.length ≤ 20
    0 ≤ grantsArray[i]
    [input] double newBudget
    [output] double
"""
def find_grants_cap(grantsArray, newBudget):
    # sort the array in a descending order.
    arr = sorted(grantsArray, reverse=True)
    print(arr)
    
    # pad the array with a zero at the end to
    # cover the case where 0 <= cap <= grantsArray[i]
    arr.append(0)
    n = len(grantsArray)
    oldBudget = 0
    for i in arr:
        oldBudget += i
    
    
    # calculate the total amount we need to
    # cut back to meet the reduced budget
    surplus = oldBudget - newBudget

    # if there is nothing to cut, simply return
    # the highest grant as the cap. Recall that
    # the grants array is sorted in a descending
    # order, so the highest grant is positioned
    # at index 0
    if surplus <= 0:
        return arr[0]
     
    # start subtracting from surplus the
    # differences (“deltas”) between consecutive
    # grants until surplus is less or equal to zero.
    # Basically, we are testing out, in order, each
    # of the grants as potential lower bound for
    # the cap. Once we find the first value that
    # brings us below zero we break
    for i in range(0, n):
        surplus = surplus - ((i+1) * (arr[i] - arr[i+1]))  ##important
        if (surplus <= 0):
            print("surplus:", surplus)
            print("arr[{}+1]: {}".format(i, arr[i+1]))
            # since arr[i+1] is a lower bound
            # to our cap, i.e. arr[i+1] <= cap,
            # we  need to add to arr[i+1] the
            # difference: (-total / float(i+1), so the
            # returned value equals exactly to cap.
            newCap = arr[i+1] + (-surplus / float(i+1))   ## important
            print("newCap = arr[{}+1] + (-{} / float({}+1))".format(i, surplus, i))
            print("newCap = {} + (-{} / float({}+1)) == {}".format(arr[i+1], surplus, float(i+1), newCap))
            
            break
    
    result_with_newCap = [] 
    for i in range(0, len(grantsArray)):
        if (grantsArray[i] > int(newCap)):
            result_with_newCap.append(newCap)
        elif (grantsArray[i] <= int(newCap)):
            result_with_newCap.append(grantsArray[i])
            
    return newCap, result_with_newCap

grantsArray = [2, 100, 50, 120, 1000]
newBudget = 190
print(find_grants_cap(grantsArray, newBudget))

In [1]:
"""
========================= Bracket Match =============================
A string of brackets is considered correctly matched if every 
opening bracket in the string can be paired up with a later closing 
bracket, and vice versa. For instance, “(())()” is correctly matched, 
whereas “)(“ and “((” aren’t. For instance, “((” could become correctly 
matched by adding two closing brackets at the end, so you’d return 2.

Given a string that consists of brackets, write a function bracketMatch
that takes a bracket string as an input and returns the minimum number
of brackets you’d need to add to the input in order to make it correctly matched.

Explain the correctness of your code, and analyze its time and space complexities.

Examples:

input:  text = “(()”
output: 1

input:  text = “(())”
output: 0

input:  text = “())(”
output: 2
Constraints:

[time limit] 5000ms

[input] string text

1 ≤ text.length ≤ 5000
[output] integer
"""


"""Only parenthesis matcher -- Maruf"""
from collections import deque
def bracketMatcher(string):
    stack = deque()
    balanced = True
    needed = 0
    for symbol in string:
        if symbol == "(":
            needed += 1
            stack.append(symbol)
        else:
            if len(stack)==0:
                needed += 1
                balanced = False
            else:
                needed -= 1
                stack.pop()

    if balanced and len(stack)==0 and needed==0:
        return string, "Balanced: YES", "NEEDED {}".format(len(stack))
    else:
        return string, "Balanced: NO", "NEEDED {}".format(len(stack))

print("Only parenthesis matcher - Maruf")
print(bracketMatcher('((()))'))  # expected 0
print(bracketMatcher('(()()'))   # expected 1
print(bracketMatcher(')('))      # expected 2
print(bracketMatcher('))))))'))  # expected 6


"""All kinds of bracket matcher"""
def allBracketMatcher(string):
    from collections import deque
    stack = deque()
    upperElement = ""
    for value in string:
        if len(stack)>0: 
            upperElement = stack[-1]
        stack.append(value)
        if (len(stack) > 1):
            if ((upperElement == '(' and stack[-1] == ')') or
                (upperElement == '{' and stack[-1] == '}') or
                (upperElement == '[' and stack[-1] == ']')):
                stack.pop()
                stack.pop()
    if len(stack)==0: 
        return string, "Balanced: YES", "NEEDED {}".format(len(stack))
    else: 
        return string, "Balanced: NO", "NEEDED {}".format(len(stack))

print("\nAll kinds of brackets matcher")
print(allBracketMatcher('{([])}'))  # expected 0
print(allBracketMatcher('({}[]'))   # expected 1
print(allBracketMatcher(')(}{]['))  # expected 6
print(allBracketMatcher(')}]]})'))  # expected 6

Only parenthesis matcher - Maruf
('((()))', 'Balanced: YES', 'NEEDED 0')
('(()()', 'Balanced: NO', 'NEEDED 1')
(')(', 'Balanced: NO', 'NEEDED 1')
('))))))', 'Balanced: NO', 'NEEDED 0')

All kinds of brackets matcher
('{([])}', 'Balanced: YES', 'NEEDED 0')
('({}[]', 'Balanced: NO', 'NEEDED 1')
(')(}{][', 'Balanced: NO', 'NEEDED 6')
(')}]]})', 'Balanced: NO', 'NEEDED 6')


In [1]:
"""
Deletion Distance - Dynamic Programming
The deletion distance of two strings is the minimum number of characters 
you need to delete in the two strings in order to get the same string. 
For instance, the deletion distance between "heat" and "hit" is 3:

By deleting 'e' and 'a' in "heat", and 'i' in "hit", we get the string "ht" in both cases.
We cannot get the same string from both strings by deleting 2 letters or fewer.
Given the strings str1 and str2, write an efficient function deletionDistance that 
returns the deletion distance between them. Explain how your function works, 
and analyze its time and space complexities.

Time Complexity:  O(N⋅M). We have a nested loop that executes O(1) 
steps at every iteration, 
thus we the time complexity is O(N⋅M) where N and M are the lengths of
str1 and str2, respectively.

Space Complexity:  O(N⋅M). We save every value of opt(i,j) in our memo 2D array, 
which takes O(N⋅M) space, where N and M are the lengths of str1 and str2, respectively.

Examples:

input:  str1 = "dog", str2 = "frog"
output: 3

input:  str1 = "some", str2 = "some"
output: 0

input:  str1 = "some", str2 = "thing"
output: 9

input:  str1 = "", str2 = ""
output: 0
"""
 
""" 
  Interviewee: 
  Ok so we can look through both strings, and delete
  chars that are not in both
  But the order matters
  so hit and heat.  If we delete e, a, i, we have th and ht (incorrect) -- TH and HT not the same
  Well what if we break down this to smaller problems
  So for heat and hit, we can solve hea and hi -- YES
  if we have h and h we can discard those, they don't impact deletion distance
  then we can look at he and hi, their deletion distance is 2 b/c e and i don't match
  then we can look at hea and hit, their deletion distance is 2 + 2 b/c a and i don't match
  ok ok, so h
  
  like this: (you're on the right track)
  
        ''    h        e                a             t
  ''    0     1        2                3             4
  h     1     0 (cc)   1(0+1for the e)  2(1+1 for a)  3 (2+1 for t)
  i     2     1        2(1+1)           3(2+1)        4(3+1) 
  t     3     2(1+1)   3                4             3(cc)
  
cc == cattycorner

		h	e	a	t       "hit", "heat": Expected 3
	0	1	2	3	4       [0, 1, 2, 3, 4]
h	1	0	1	2	3       [1, 0, 1, 2, 3]
i	2	1	2	3	4       [2, 1, 2, 3, 4]
t	3	2	3	4	3       [3, 2, 3, 4, 3]

"""
def deletionDistance(str1, str2):
    str1Len = len(str1)
    str2Len = len(str2)
    
    # allocate a 2D array with str1Len + 1 rows and str2Len + 1 columns
    memo = [[0 for col in range(str2Len+1)] for row in range(str1Len+1)]
    for row in memo: print(row)
    print()

    for i in range(0, str1Len+1):
        for j in range(0, str2Len+1):
            if (i == 0):
                memo[i][j] = j  # Rule #1: This is true because if one string is the empty string, 
                                # we have no choice but to delete all letters in the other string.
            elif (j == 0):
                memo[i][j] = i  # Same as Rule #1 
            
            # Rule #2: This holds since we don’t need to delete the last 
            # letters in order to get the same string, we simply
            # use the same deletions we would to the (i-1)'th and 
            # (j-1)'th prefixes.
            elif (str1[i-1] == str2[j-1]):        
                memo[i][j] = memo[i-1][j-1] 
                                                  
            else:
                # Rule #3: This holds since we need to delete at least one
                # of the letters str1[i] or str2[j] 
                # and the deletion of one of the letters is counted as 1
                # deletion (hence the 1 in the 
                # formula). Then, since we’re left with either 
                # the (i-1)'th prefix of str1, or the
                # (j-1)'th prefix of str2, need to take the 
                # minimum between opt(i-1,j) and opt(i,j-1).
                # We, therefore, get the equation 
                # opt(i,j) = 1 + min(opt(i-1,j), opt(i,j-1)).
                memo[i][j] = 1 + min(memo[i-1][j], memo[i][j-1])

    for i in memo:
        print(i)
    return memo[str1Len][str2Len]

print("case 1: \"\", \"\" Expected 0")
print("result: ", deletionDistance("", ""))
print("case 2: \"\", hit Expected 3")
print("result: ", deletionDistance("", "hit"))
print("case 3: neat, \"\" Expected 4")
print("result: ", deletionDistance("neat", ""))
print("case 4: heat, hit Expected 3")
print("result: ", deletionDistance("heat", "hit"))
print("case 5: hot, not Expected 2")
print("result: ", deletionDistance("hot", "not"))
print("case 6: some, thing Expected 9")
print("result:", deletionDistance("some", "thing"))
print("case 7: abc, adbc Expected 1")
print("result: ", deletionDistance("abc", "adbc"))
print("case 8: awesome, awesome Expected 0")
print("result: ", deletionDistance("awesome", "awesome"))
print("case 9: ab, ba Expected 2")
print("result: ", deletionDistance("ab", "ba"))
print("case 10: hit, heat Expected 3")
print("result: ", deletionDistance("hit", "heat"))

case 1: "", "" Expected 0
[0]

[0]
result:  0
case 2: "", hit Expected 3
[0, 0, 0, 0]

[0, 1, 2, 3]
result:  3
case 3: neat, "" Expected 4
[0]
[0]
[0]
[0]
[0]

[0]
[1]
[2]
[3]
[4]
result:  4
case 4: heat, hit Expected 3
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]

[0, 1, 2, 3]
[1, 0, 1, 2]
[2, 1, 2, 3]
[3, 2, 3, 4]
[4, 3, 4, 3]
result:  3
case 5: hot, not Expected 2
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 0, 0]

[0, 1, 2, 3]
[1, 2, 3, 4]
[2, 3, 2, 3]
[3, 4, 3, 2]
result:  2
case 6: some, thing Expected 9
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]

[0, 1, 2, 3, 4, 5]
[1, 2, 3, 4, 5, 6]
[2, 3, 4, 5, 6, 7]
[3, 4, 5, 6, 7, 8]
[4, 5, 6, 7, 8, 9]
result: 9
case 7: abc, adbc Expected 1
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]
[0, 0, 0, 0, 0]

[0, 1, 2, 3, 4]
[1, 0, 1, 2, 3]
[2, 1, 2, 1, 2]
[3, 2, 3, 2, 1]
result:  1
case 8: awesome, awesome Expected 0
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 

In [ ]:
"""
MINIMUM NUMBER OF OPERATIONS TO TRANSFORM ONE STRING TO ANOTHER
# Dynamic Programming implementation to find minimum number of deletions and insertions
Source: https://www.geeksforgeeks.org/minimum-number-deletions-insertions-transform-one-string-another/
"""

# Returns length of length common subsequence for str1[0..m-1], str2[0..n-1]
def transform_with_min_ops(str1, str2, m, n):
    memo = [[0 for col in range(n + 1)]for row in range(m + 1)]
    for row in memo: print(row)
    # Following steps build memo[m+1][n+1] in bottom up fashion. 
    # Note that memo[i][j] contains length of LCS of str1[0..i-1] and str2[0..j-1]
    for i in range(m + 1):
        for j in range(n + 1):
            if (i == 0 or j == 0):
                memo[i][j] = 0
            elif (str1[i-1] == str2[j - 1]):
                memo[i][j] = 1 + memo[i - 1][j - 1]
            else:
                memo[i][j] = max(memo[i - 1][j], memo[i][j - 1])

    # memo[m][n] contains length of LCS for X[0..n-1] and Y[0..m-1]
    print()
    for row in memo: print(row)
    return memo[m][n]

def printMinDelAndInsert(str1, str2):
    """ Function to find minimum number of deletions and insertions """
    m = len(str1)
    n = len(str2)
    length = transform_with_min_ops(str1, str2, m, n)
    print("\nTo transform '" + str1 + "' to '" + str2 + "':")
    print("\tMinimum number of deletions from '{}' = {}".format(str1, m - length))
    print("\tMinimum number of insertions from '{}' = {}".format(str2, n - length))
    
# Driver Code
printMinDelAndInsert("heap", "pea")
printMinDelAndInsert("pea", "heap")
printMinDelAndInsert("", "")
printMinDelAndInsert("", "hit")
printMinDelAndInsert("neat", "")
printMinDelAndInsert("heat", "hit")
printMinDelAndInsert("hit", "heat")
printMinDelAndInsert("hot", "not")
printMinDelAndInsert("some", "thing")
printMinDelAndInsert("thing", "some")
printMinDelAndInsert("abc", "adbc")
printMinDelAndInsert("adbc", "abc")
printMinDelAndInsert("awesome", "awesome")
printMinDelAndInsert("ab", "ba")
printMinDelAndInsert("ba", "ab")

In [ ]:
"""
Merging 2 Packages
Given a package with a weight limit limit and an array arr of item weights, implement a function
get_indices_of_item_wights() that --
    1. Finds two items whose sum of weights equals the weight limit limit.
    2. Your function should return a pair [i, j] of the indices of the item weights, ordered such that i > j. 
    3. If such a pair doesn’t exist, return an empty array.
Analyze the time and space complexities of your solution.

Time Complexity: going over the array only once, performing constant time work for each weight and 
                assuming we have a good hash function with rare collisions, 
                we get a linear O(N) time complexity.
Space Complexity: O(N).
"""
def get_indices_of_item_wights(arr, limit):
    memo = {}
    for i in range(0, len(arr)):
        print(memo)
        if arr[i] in memo:
            return sorted([memo[arr[i]],i], reverse=True)
        else:
            memo[limit-arr[i]] =  i
    return []

def get_all_indices_and_value_pairs_of_item_wights(arr, limit):
    indices_pairs = []
    value_pairs = []
    memo = {}
    
    for i in range(0, len(arr)):
        if arr[i] in memo:
            indices_pairs.append(sorted([memo[arr[i]],i], reverse=True))         
            value_pairs.append(sorted([arr[memo[arr[i]]],arr[i]], reverse=True))
            ## if you want incremental/unsorted, simply remove the sorted() method from above:
        else:
            memo[limit-arr[i]] =  i
    for i in range(len(indices_pairs)):
        print("{a}:{b}\t== {c}".format(a=indices_pairs[i], b=value_pairs[i], c=limit))
        
arr, limit = [4, 15, 9, 18, 7, 16, 5, 23, 2, 10], 25
print("Input array:\t {k} ;\texpected:\t {a}".format(k=arr, a="[4, 3]"))
print("get_indices_of_item_wights:", get_indices_of_item_wights(arr, limit))
print("get_all_indices_and_value_pairs_of_item_wights: ")
get_all_indices_and_value_pairs_of_item_wights(arr, limit)
arr, limit = [9], 9  # expected []
result = get_indices_of_item_wights(arr, limit)
print("\nInput array:{k};\tlimit:{m};\texpected:\t {a};\t actual:\t {b}".format(k=arr, m=limit, a="[]", b=result))
arr, limit = [4,4], 8  # expected [1,0]
result = get_indices_of_item_wights(arr, limit)
print("Input array:{k};\tlimit:{m};\texpected:\t {a};\t actual:\t {b}".format(k=arr, m=limit, a="[1,0]", b=result))
arr, limit = [4,4,1], 5  # expected [2,1]
result = get_indices_of_item_wights(arr, limit)
print("Input array:{k};\tlimit:{m};\texpected:\t {a};\t actual:\t {b}".format(k=arr, m=limit, a="[2,1]", b=result))
arr, limit = [4,6,10,15,16], 21  # expected [3,1]
result = get_indices_of_item_wights(arr, limit)
print("Input array:{k};\tlimit:{m};\texpected:\t {a};\t actual:\t {b}".format(k=arr, m=limit, a="[3,1]", b=result))
arr, limit = [4,6,10,15,16], 20   # expected [4,0]
result = get_indices_of_item_wights(arr, limit)
print("Input array:{k};\tlimit:{m};\texpected:\t {a};\t actual:\t {b}".format(k=arr, m=limit, a="[4,0]", b=result))
arr, limit = [12,6,7,14,19,3,0,25,40], 7 #expected [6,2]
result = get_indices_of_item_wights(arr, limit)
print("Input array:{k};\tlimit:{m};\texpected:\t {a};\t actual:\t {b}".format(k=arr, m=limit, a="[6,2]", b=result))


In [2]:
""" Array of Array Products
Given an array of integers arr, you’re asked to calculate for each index
i the product of all integers except the integer at that index (i.e. except arr[i]).
Implement a function arrayOfArrayProducts that takes an array of integers and returns an array of the products.

Solve without using division and analyze your solution’s time and space complexities.

Examples:

input:  arr = [8, 10, 2]
output: [20, 16, 80] # by calculating: [10*2, 8*2, 8*10]

input:  arr = [2, 7, 3, 4]
output: [84, 24, 56, 42] # by calculating: [7*3*4, 2*3*4, 2*7*4, 2*7*3]
Constraints: 0 ≤ arr.length ≤ 20
"""

def calcProductArray_brute_pramp(arr): #brute force pramp version O(N^2), O(N)
    n = len(arr)
    if (n == 0 or n == 1):
        # nothing to multiply if n equals to 0 or 1
        return arr

    productArr = []
    for i in range(0, n):
        product = 1
        for j in range(0, n):
            if(i != j):
                product *= arr[j]

        productArr.append(product)

    return productArr

def findProduct(arr):  # O(N), O(N)
    left = 1
    product = []
    for ele in arr:
        product.append(left)
        left = left * ele
    print("interim: ", product)
    
    right = 1
    for i in range(len(arr)-1,-1,-1):
        product[i] = product[i] * right
        right = right * arr[i]
    
    return product

#Test Programs
def test():
    arr = [8, 10, 2]                     
    expected = [20, 16, 80]
    print("given array: ", arr)
    print("Expected:    \t", expected)
#     print("Found(brute_maruf):\t", calcProductArray_brute(arr))
#     print("Found(brute_pramp):\t", calcProductArray_brute_pramp(arr))
    print("findProduct:\t", findProduct(arr))

    arr2 = [2, 7, 3, 4]                  
    expected = [84, 24, 56, 42]
    print("given array: ", arr2)
    print("Expected:    \t", expected)
#     print("Found(brute_maruf):\t", calcProductArray_brute(arr2))
#     print("Found(brute_pramp):\t", calcProductArray_brute_pramp(arr2))
    print("findProduct:\t", findProduct(arr2))

    arr3 = [-3,17,430,-6,5,-12,-11,5]    
    expected = [-144738000,25542000,1009800,-72369000,86842800,-36184500,-39474000,86842800]
    print("given array: ", arr3)
    print("Expected:    \t", expected)
#     print("Found(brute_maruf):\t", calcProductArray_brute(arr3))
#     print("Found(brute_pramp):\t", calcProductArray_brute_pramp(arr3))
    print("findProduct:\t", findProduct(arr3))

    arr4 = []    
    expected = []
    print("given array: ", arr4)
    print("Expected:    \t", expected)
#     print("Found(brute_maruf):\t", calcProductArray_brute(arr4))
#     print("Found(brute_pramp):\t", calcProductArray_brute_pramp(arr4))
    print("findProduct:\t", findProduct(arr4))

    arr5 = [5]    
    expected = []
    print("given array: ", arr5)
    print("Expected:    \t", expected)
#     print("Found(brute_maruf):\t", calcProductArray_brute(arr5))
#     print("Found(brute_pramp):\t", calcProductArray_brute_pramp(arr5))
    print("findProduct:\t", findProduct(arr5))
    
    arr6 = [2,3,0,982,10]
    expected = [0,0,58920,0,0]
    print("given array: ", arr6)
    print("Expected:    \t", expected)
#     print("Found(brute_maruf):\t", calcProductArray_brute(arr6))
#     print("Found(brute_pramp):\t", calcProductArray_brute_pramp(arr6))
    print("findProduct:\t", findProduct(arr6))

    arr7 = [2,2]
    expected = [2,2]
    print("given array: ", arr7)
    print("Expected:    \t", expected)
#     print("Found(brute_maruf):\t", calcProductArray_brute(arr7))
#     print("Found(brute_pramp):\t", calcProductArray_brute_pramp(arr7))
    print("findProduct:\t", findProduct(arr7))

test()

given array:  [8, 10, 2]
Expected:    	 [20, 16, 80]
interim:  [1, 8, 80]
findProduct:	 [20, 16, 80]
given array:  [2, 7, 3, 4]
Expected:    	 [84, 24, 56, 42]
interim:  [1, 2, 14, 42]
findProduct:	 [84, 24, 56, 42]
given array:  [-3, 17, 430, -6, 5, -12, -11, 5]
Expected:    	 [-144738000, 25542000, 1009800, -72369000, 86842800, -36184500, -39474000, 86842800]
interim:  [1, -3, -51, -21930, 131580, 657900, -7894800, 86842800]
findProduct:	 [-144738000, 25542000, 1009800, -72369000, 86842800, -36184500, -39474000, 86842800]
given array:  []
Expected:    	 []
interim:  []
findProduct:	 []
given array:  [5]
Expected:    	 []
interim:  [1]
findProduct:	 [1]
given array:  [2, 3, 0, 982, 10]
Expected:    	 [0, 0, 58920, 0, 0]
interim:  [1, 2, 6, 0, 0]
findProduct:	 [0, 0, 58920, 0, 0]
given array:  [2, 2]
Expected:    	 [2, 2]
interim:  [1, 2]
findProduct:	 [2, 2]


In [13]:
"""
Given a 2D array (matrix) inputMatrix of integers, create a function spiralCopy that copies
inputMatrix’s values into a 1D array in a spiral order, clockwise. Your function then should 
return that array. Analyze the time and space complexities of your solution.
See the illustration below to understand better what a clockwise spiral order looks like.

Pseudo: 
4 for loops
1 for loop to get top row, left to right
1 for loop to get left col, top to bottom
1 for loop to get bottom row, right to left
1 for loop to get right col, bottom to top
"""

def spiral_copy(inputMatrix):
    result = []
    if len(inputMatrix) == 0:
        return result
    elif len(inputMatrix) == 1:
        for col in range(0, len(inputMatrix[0])):
            result.append(inputMatrix[0][col])
        return result
    elif len(inputMatrix[0]) == 1:
        for row in range(0, len(inputMatrix)):
            result.append(inputMatrix[row][0])
        return result
    else:
        top_row = 0
        bottom_row = len(inputMatrix) - 1
        left_col = 0
        right_col = len(inputMatrix[top_row])-1
        while (top_row <= bottom_row) and (left_col <= right_col):  
            for col in range(left_col, right_col+1):         # move top left to top right
                result.append(inputMatrix[top_row][col])
            top_row += 1
            
            for row in range(top_row, bottom_row+1):         # move top right to down right
                result.append(inputMatrix[row][right_col])
            right_col = right_col - 1
            
            for col in range(right_col, left_col-1, -1):     # move down right to down left
                result.append(inputMatrix[bottom_row][col])
            bottom_row = bottom_row -1
            
            for row in range(bottom_row, top_row-1, -1):     # move down left to top left
                result.append(inputMatrix[row][left_col])
            left_col = left_col + 1
    return result      

print(spiral_copy([[1]]))     # expected [1]
print(spiral_copy([[1],[2]])) # expected [1,2]
print(spiral_copy([[1,2]]))   # expected [1,2]
print(spiral_copy([[1,2],[3,4]])) # expected [1,2,4,3]
print(spiral_copy([[1,2,3,4,5],[6,7,8,9,10]])) #expected [1,2,3,4,5,10,9,8,7,6]
print(spiral_copy([[1,2,3,4,5],[6,7,8,9,10],[11,12,13,14,15],[16,17,18,19,20]]))
#expected [1,2,3,4,5,10,15,20,19,18,17,16,11,6,7,8,9,14,13,12]
  

[1]
[1, 2]
[1, 2]
[1, 2, 4, 3]
[1, 2, 3, 4, 5, 10, 9, 8, 7, 6]
[1, 2, 3, 4, 5, 10, 15, 20, 19, 18, 17, 16, 11, 6, 7, 8, 9, 14, 13, 12]


In [ ]:
""" Smallest Substring of All Characters -- KIRAN (recursive solution)
Given an array of unique characters arr and a string str, Implement a function 
getShortestUniqueSubstring that finds the smallest substring of str containing all the 
characters in arr. Return "" (empty string) if such a substring doesn’t exist.

Come up with an asymptotically optimal solution and analyze the time and space complexities.

Example:
input:  arr = ['x','y','z'], str = "xyyzyzyx"
output: "zyx"
"""

def get_shortest_unique_substring(arr, str):
    if not str:
        return ""
    if len(arr) == 1:
        if arr[0] == str:
            return str
        else:
            return ""
    res = [helper(arr, str[i:]) for i in range(len(str) - 1)]
    res = [x for x in res if x]

    result = ''
    max_len = len(str)
    #print(res)
    for elem in res:
        if len(elem) <= max_len:
            max_len = len(elem)
            result = elem
    return result

def helper(arr, string):
    from collections import defaultdict
    # print(arr, string)
    count = len(arr)
    seen = defaultdict(int)
    subString = []
    for c, i in enumerate(string):
        if i in arr:
            seen[i] += 1
            # print(seen)
            if len(seen) == count:
                # print(seen, string[:c+1])
                return string[:c + 1]

    return ''

print("Result:", get_shortest_unique_substring(["A"], "" )) # Expected: ""
print("Result:", get_shortest_unique_substring(["A"], "B" )) # Expected: ""
print("Result:", get_shortest_unique_substring(["A"], "A" )) # Expected: "A"
print("Result:", get_shortest_unique_substring(["A","B","C"], "ADOBECODEBANCDDD" )) # Expected: "BANC"
print("Result:", get_shortest_unique_substring(["A","B","C","E","K","I"], "KADOBECODEBANCDDDEI" )) # Expected: "KADOBECODEBANCDDDEI"
print("Result:", get_shortest_unique_substring(["x","y","z"], "xyyzyzyx" )) # Expected: "zyx"
print("Result:", get_shortest_unique_substring(["x","y","z","r"], "xyyzyzyx" )) # Expected: ""

In [ ]:
"""
Island Count
Given a 2D array binaryMatrix of 0s and 1s, implement a function getNumberOfIslands that returns the number of islands of 1s in binaryMatrix.

An island is defined as a group of adjacent values that are all 1s. A cell in binaryMatrix is considered 
adjacent to another cell if they are 
next to each either on the same row or column. Note that two values of 1 are not part of the same island
if they’re sharing only a mutual “corner” (i.e. they are diagonally neighbors).

Explain and code the most efficient solution possible and analyze its time and space complexities.

Example:

input:  binaryMatrix = [ [0,    1,    0,    1,    0],
                         [0,    0,    1,    1,    1],
                         [1,    0,    0,    1,    0],
                         [0,    1,    1,    0,    0],
                         [1,    0,    1,    0,    1] ]

output: 6 

Solution:
To solve this problem, we’ll traverse binaryMatrix and every time we come across a cell of 1 
we’ll do the following: Change that cell and all its vertically and horizontally (but not diagonally)
adjacent 1s into -1s. We do this “expansion” in order to avoid recounting of islands. 
Increment islands - which is our counter for number of islands - by 1. Expanding from a cell whose 
value is 1 to other adjacent 1s in binaryMatrix is similar to running a Breadth-First Search (BFS) 
or a Depth-First Search (DFS).
In our case, we’ll avoid using a recursion and instead opt for an iterative approach to expand to
all adjacent 1s. We do so by using queue that holds the next indices to visit. We keep expanding to
other adjacent 1s as long as the queue is not empty. Whenever we encounter a value of -1 in our traversal,
we ignore it since it is part on an island we’ve already counted.

Time Complexity: let N and M be the numbers of columns and rows in binaryMatrix, respectively. 
Each cell in binaryMatrix is visited a constant number of times. Once during the iteration and 
up to 4 times during an island expansion. Therefore, the time complexity is linear in the size 
of the input, i.e. O(N⋅M).
Space Complexity: since we are allocating a queue in the algorithm, the space complexity is 
linear O(N⋅M). For instance, consider a matrix that is all 1s.
"""
def getNumberOfIslands(binaryMatrix):
    print("\ninput matrix: ") #printing the input
    for row in binaryMatrix:
        print(row)
        
    islands = 0
    rows = len(binaryMatrix)    # number of rows
    cols = len(binaryMatrix[0]) # number of columns
    for i in range(0, rows):
        for j in range(0, cols):
            if (binaryMatrix[i][j] == 1):
                markIsland(binaryMatrix, rows, cols, i, j)
                islands += 1  
                
    print("number of islands: ", islands)
    return islands

def markIsland(binaryMatrix, rows, cols, i, j):
    from collections import deque
    q = deque()   #queue
    q.append([i,j])
    while len(q)>0:
        item = q.popleft()
        x = item[0]
        y = item[1]
        if (binaryMatrix[x][y] == 1):
            binaryMatrix[x][y] = -1
            pushIfValid(q, rows, cols, x-1, y)
            pushIfValid(q, rows, cols, x, y-1)
            pushIfValid(q, rows, cols, x+1, y)
            pushIfValid(q, rows, cols, x, y+1)

def pushIfValid(q, rows, cols, x, y):
    if (x >= 0 and x < rows and y >= 0 and y < cols):
        q.append([x,y])      

# getNumberOfIslands([[0]]) # Expected: 0
# getNumberOfIslands([[1]]) # Expected: 1
# getNumberOfIslands([[1,0,1,0]]) # Expected: 2
# getNumberOfIslands([[1,0,1,0],[0,1,1,1],[0,0,1,0]]) # Expected: 2
# getNumberOfIslands([[1,0,1,0],[0,1,1,1],[0,0,1,0],[1,1,0,0],[0,1,0,1]]) # Expected: 4
getNumberOfIslands([[0,1,0,1,0],[0,0,1,1,1],[1,0,0,1,0],[0,1,1,0,0],[1,0,1,0,1]]) # Expected: 6
# getNumberOfIslands([[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1]]) # Expected: 1

In [10]:
"""Sales Path
The car manufacturer Honda holds their distribution system in the form of a tree (not necessarily binary).
The root is the company itself, and every node in the tree represents a car distributor that receives cars 
from the parent node and ships them to its children nodes. The leaf nodes are car dealerships that sell 
cars direct to consumers. In addition, every node holds an integer that is the cost of shipping a car to it.
A path from Honda’s factory to a car dealership, which is a path from the root to a leaf in the tree, is 
called a Sales Path. The cost of a Sales Path is the sum of the costs for every node in the path. 
For example, in the tree above one Sales Path is 0→3→0→10, and its cost is 13 (0+3+0+10).
Honda wishes to find the minimal Sales Path cost in its distribution tree. Given a node rootNode, 
write a function getCheapestCost that calculates the minimal Sales Path cost in the tree.
Take for example the tree below:
         0
     /   |   \
   5     3     6
  /     / \   / \
 4     2   0  1  5
      /   /
      1  10
       \
        1
Implement your function in the most efficient manner and analyze its time and space complexities.
For example:
Given the rootNode of the tree in diagram above
Your function would return: 7 since it’s the minimal Sales Path cost (there are actually two 
Sales Paths in the tree whose cost is 7: 0→6→1 and 0→3→2→1→1)

Obviously iterating through all paths again and again is not a good solution, since its wasteful
in terms of time and memory. But intuitively if we find a solution that uses previous calculations 
somehow. This hints that the solution should involve recursion in some manner.
First we notice that if the root is also a leaf, the best Sales Path, is simply the value in the 
node itself. This is the base case for the solution. If the root has children, then the minimal 
Sales Path is also a minimal path from the root’s child. Thus, if we already know the minimal cost
for the root’s children, then the minimal cost for the root is simply the minimum of the values for
its children plus the value stored in the root itself.

Time Complexity:
let N be the number of nodes in the tree. Notice that getCheapestCost is applied
to every node exactly once. Therefore, there are overall O(N) calls to getCheapestCost.
Space Complexity: 
every time the function recurses, it consumes only a constant amount of space. 
However, due to the nature of the recursion we used, the stack call holds N instances of 
getCheapestCost which makes the total space complexity to be O(N).
"""

# A node 
class Node:
  # Constructor to create a new node
  def __init__(self, cost):
    self.cost = cost
    self.children = []
    self.parent = None
    
def getCheapestCost(rootNode): # bottom up approach
    n = len(rootNode.children)
    if (n == 0): 
        return rootNode.cost
    else:
        # initialize minCost to the largest integer in the system
        minCost = float("inf")  #maximun value for an int
        for i in range(0, n):
            tempCost = getCheapestCost(rootNode.children[i])
            if (tempCost < minCost):
                minCost = tempCost
    return minCost + rootNode.cost

# def getCheapestCost(rootNode): # top down up approach
#     """ Traverse all nodes, accumalte the sum of visited nodes
#         hold the minimum value when reaching a leaf one.
#         time complexity: O(N) where N is the number of nodes.
#     """   
#     minValue = float("inf")
#     sumSoFar = 0
    
#     def getCheapestCostRecursive(rootNode, sumSoFar, minValue):
#         if rootNode == None:
#             minValue = min(minValue, sumSoFar)
#             return
    
#         if sumSoFar >= minValue:
#             return
          
#         for i in range(len(rootNode.children)):
#             getCheapestCostRecursive(
#                 rootNode.children[i], 
#                 sumSoFar + rootNode.children[i].cost,
#                 minValue
#             )
    
#     getCheapestCostRecursive(rootNode, sumSoFar, minValue)
#     return minValue
    

"""=========TESTING==========
         0
     /   |   \
   5     3     6
  /     / \   / \
 4     2   0  1  5
      /   /
      1  10
       \
        1
"""
node0 = Node(0) 
node0.children = [Node(10)]
node1 = Node(1) 
node1.children = [Node(1)]
node2 = Node(2) 
node2.children = [node1]
node5 = Node(5) 
node5.children = [Node(4)]
node3 = Node(3) 
node3.children = [node2, node0]
node6 = Node(6) 
node6.children = [Node(1),Node(5)]
root = Node(0)
root.children = [node5, node3, node6]
print("\nresult: ", getCheapestCost(root)) # expected 7


result:  inf


In [ ]:
"""alternate approach WIP"""
def deletion_distance(str1, str2):
    sub_string_dist = []
    for i in range(len(str1)):
        row = [0] * (len(str2))
        sub_string_dist.append(row)

    for i in range(len(str1)):
        for j in range(len(str2)):
            # if 0 row or 0 col, dist is from empty string
            if i == 0:
                sub_string_dist[i][j] = j  # argh == instead of =

            elif j == 0:
                sub_string_dist[i][j] = i
    # ok the above is good
    print(sub_string_dist)
    # my indexes are off by 1.
    # it's not accounting for the 'empty' str at the beginning

    for i in range(len(str1)):
        for j in range(len(str2)):
            # if equal, take cattycorner
            if str1[i] == str2[j]:
                sub_string_dist[i][j] = sub_string_dist[i - 1][j - 1]
            # if not, add one to min from adjacent
            elif str1[i] != str2[j]:
                sub_string_dist[i][j] = 1 + min(sub_string_dist[i - 1][j], sub_string_dist[i][j - 1])

    return sub_string_dist  # you should return the proper value from here, 
    # not the whole 2D array. once done, please run tests


def test():
    str1 = 'dog'
    str2 = 'frog'
    print(deletion_distance(str1, str2))


test() 

In [7]:
"""Note"""
s = 'ABCDEFGH'
res= [s[i:] for i in range(len(s)-1)]
print(res)
res = [x for x in res if x]
print(res)

['ABCDEFGH', 'BCDEFGH', 'CDEFGH', 'DEFGH', 'EFGH', 'FGH', 'GH']
['ABCDEFGH', 'BCDEFGH', 'CDEFGH', 'DEFGH', 'EFGH', 'FGH', 'GH']


In [ ]:
"""
Decrypt Message

Messages consist of lowercase latin letters only, and every word is encrypted separately as follows:
Convert every letter to its ASCII value. Add 1 to the first letter, and then for every letter from the second
one to the last one, add the value of the previous letter. Subtract 26 from every letter until it is in the
range of lowercase letters a-z in ASCII. Convert the values back to letters.
For instance, to encrypt the word “crime”
Decrypted message:	
        c	r	i	m	e
Step 1:	99	114	105	109	101
Step 2:	100	214	319	428	529
Step 3:	100	110	111	116	113
Encrypted message:	
        d	n	o	t	q
        
Write a function named decrypt(word) that receives a string that consists of small latin letters only, 
and returns the decrypted word.

Examples:
input:  word = "dnotq"
output: "crime"
input:  word = "flgxswdliefy"
output: "encyclopedia"

First of all, notice that the first letter is very easy to decrypt:
Convert the first letter back to its ASCII value.
Subtract 1 from it.
Move the value to be in the range of a-z ASCII values (97-122), by adding 26.
Convert the result back to a character.
The decryption of the rest of the letters is done by almost the same algorithm - given the decrypted 
previous letter prev, and its value after the second step of encryption - denoted secondStepPrev:
Convert the current letter back to its ASCII value.
Subtract secondStepPrev from it.
Move the value to be in the range of a-z ASCII value (97-122), by adding multiples of 26.
Convert the result back to a character. Store its ASCII value in prev, and add its value to secondStepPrev
(for the decryption of the next letter).
Let’s examine the algorithm using the following notation:
dec[n] - the n’th letter before encryption.
enc[n] - the n’th letter after encryption.
secondStep[n] - the n’th letter immediately after step 2 in the encryption.
The encryption algorithm gives the following relation for some integer m (which represents the number of times 
we need to add 26 to get to an ascii value):
enc[n] = dec[n] + secondStep[n-1] + 26m
By isolating dec[n], we get:
dec[n] = enc[n] - secondStep[n-1] - 26m
Though the value of m isn’t initially known, since the value of the decrypted letter must be in the ASCII range
of a-z, the decrypted letter is easily found adding 26’s to enc[n] - secondStep[n] until it is in the right range.

Time Complexity: the function’s asymptotic time complexity is O(N), where N is the length of the input string. 
the loop that iterates through the letters in the input is performed N times. In the loop, almost every step is
done in O(1), except for the loop that is supposed to move the decrypted letter back to the range of a-z.
Theoretically, the secondStep may grow linearly with the size of the input. There are two ways to deal with this:
Instead of secondStep itself, we may only keep its remainder after being divided by 26 (since we add/subtract 
multiples of 26 anyway, the equation dec[N] = enc[N] - (secondStep[N-1] % 26)- 26M still holds, only for a 
different M). This way all values in every iteration are kept in a constant range.
Note that since in practice this function is used only for words in the English language, the input is bounded 
and we therefore may ignore the growth of the secondStep anyway.

Space Complexity: the space usage is also O(N) since the output is the same size of the input, and we only keep 
the output and the second step in storage.
"""
def encrypt(word):
  """
  input:  word = "crime"
  output: "dnotq"

  input:  word = "encyclopedia"
  output: "flgxswdliefy"
  
  input: ""
  output: ""
  """
  secondStep = 1
  encrypted = ""
  
  for i in range(0, len(word)):
    asci = ord(word[i]) + secondStep
    
    while (asci > ord('z')):
      asci -= 26
      
    encrypted += chr(asci)
    secondStep = asci
    
  return encrypted

def decrypt(word):
  """
  input:  word = "dnotq"
  output: "crime"

  input:  word = "flgxswdliefy"
  output: "encyclopedia"
  """
  secondStep = 1
  decrypted = ""
  
  for i in range(0, len(word)):
    asci = ord(word[i]) - secondStep
    
    while (asci < ord('a')):
      asci += 26
      
    decrypted += chr(asci)
    secondStep = ord(word[i])
    
  return decrypted

print(encrypt("crime"))
print(encrypt("encyclopedia"))
print(decrypt("dnotq"))
print(decrypt("flgxswdliefy"))

In [11]:
"""
Time Planner
Implement a function meetingPlanner that given the availability, slotsA and slotsB, of two people and a meeting 
duration dur, returns the earliest time slot that works for both of them and is of duration dur. If there is no 
common time slot that satisfies the duration requirement, return an empty array.
Time is given in a Unix format called Epoch, which is a nonnegative integer holding the number of seconds that
have elapsed since 00:00:00 UTC, Thursday, 1 January 1970.
Each person’s availability is represented by an array of pairs. Each pair is an epoch array of size two. 
The first epoch in a pair represents the start time of a slot. The second epoch is the end time of that slot. 
The input variable dur is a positive integer that represents the duration of a meeting in seconds. 
The output is also a pair represented by an epoch array of size two.
In your implementation assume that the time slots in a person’s availability are disjointed, i.e, time slots 
in a person’s availability don’t overlap. Further assume that the slots are sorted by slots’ start time.
Implement an efficient solution and analyze its time and space complexities.

A naive solution would loop through both input arrays and check the intersection of every possible pair slots 
to find an overlap of at least dur seconds. This isn’t an efficient solution and its time complexity is O(N⋅M). 
We can do better than that.

Since the arrays are sorted by the slots’ start times, we can iterate over both arrays in a single loop.
We use two indices, one for each array, while incrementing one index at a time according the following rules:
If there is a minimal overlap of dur between two given times slots, return the pair [start, start + dur], 
where start is the start time of said overlap. Otherwise, increment the index of the array with the earlier
time slot.

Time Complexity: we are traversing every input array at most once, hence the time complexity is linear, 
i.e O(N+M), where N and N are lengths of slotsA and slotsB, respectively.

Space Complexity: it’s O(1). We are using four auxiliary variables, all of which are occupying only a
constant amount of space.
"""
def meeting_planner(slotA, slotB, duration):
    result = []
    i = 0
    j = 0
    lenSlotA = len(slotA)
    lenSlotB = len(slotB)
    print("\nslotA = {}, slotB = {}, duration = {}".format(slotA, slotB, dur))
    print("\ti = {}, j = {} ".format(i, j))
    while i < lenSlotA and j < lenSlotB:
        maxStart = max(slotA[i][0], slotB[j][0])
        minEnd = min(slotA[i][1], slotB[j][1])
        
        desiredStart = maxStart
        desiredEnd = maxStart + duration
        
        print("slotA = {}, slotB = {}".format(slotA[i], slotB[j]))
        print("\tdesiredStart, desiredEnd: ", desiredStart, desiredEnd)
    
        if desiredEnd > minEnd:
            if desiredEnd > slotA[i][1]:
                i += 1 
            elif desiredEnd > slotB[j][1]:
                j += 1
        else:
            result = [desiredStart, desiredEnd]
            break;
        print("\ti = {}, j = {} ".format(i, j))
    
    return result

slotsA = [[10, 50], [60, 120], [140, 210]]
slotsB = [[0, 15], [60, 70]]
dur = 8
print("result: ", meeting_planner(slotsA, slotsB, dur))

slotsA = [[10, 50], [60, 120], [140, 210]]
slotsB = [[0, 15], [60, 70]]
dur = 12
print("result: ", meeting_planner(slotsA, slotsB, dur))


slotA = [[10, 50], [60, 120], [140, 210]], slotB = [[0, 15], [60, 70]], duration = 8
	i = 0, j = 0 
slotA = [10, 50], slotB = [0, 15]
	desiredStart, desiredEnd:  10 18
	i = 0, j = 1 
slotA = [10, 50], slotB = [60, 70]
	desiredStart, desiredEnd:  60 68
	i = 1, j = 1 
slotA = [60, 120], slotB = [60, 70]
	desiredStart, desiredEnd:  60 68
result:  [60, 68]

slotA = [[10, 50], [60, 120], [140, 210]], slotB = [[0, 15], [60, 70]], duration = 12
	i = 0, j = 0 
slotA = [10, 50], slotB = [0, 15]
	desiredStart, desiredEnd:  10 22
	i = 0, j = 1 
slotA = [10, 50], slotB = [60, 70]
	desiredStart, desiredEnd:  60 72
	i = 1, j = 1 
slotA = [60, 120], slotB = [60, 70]
	desiredStart, desiredEnd:  60 72
	i = 1, j = 2 
result:  []


In [37]:
"""
Array Quadruplet
Given an unsorted array of integers arr and a number s, write a function findArrayQuadruplet that finds four numbers
(quadruplet) in arr that sum up to s. Your function should return an array of these numbers in an ascending order. 
If such a quadruplet doesn’t exist, return an empty array.
Note that there may be more than one quadruplet in arr whose sum is s. You’re asked to return the first one you 
encounter (considering the results are sorted).

Solution:
The naive solution would be to consider every quadruplet in the input array and return the one (if exists) 
whose sum is s. This approach requires using 4 nested loops and its time complexity is O(N^4). This is quite 
inefficient and we can do better than that.
We start by sorting the given array in ascending order and then for each pair (arr[i], arr[j]) in the array 
where (i < j), we check if a quadruplet is formed by current pair and a pair from a subarray arr[j+1...n-1]. 
So how do we find a complementing pair in the subarray arr[j+1...n-1]?
What we want to do is to find two values in the subarray such that their sum equals to s - (arr[i], arr[j]). 
Let’s denote this value as r. Now, since we made sure to sort arr in an ascending order, the idea is to maintain
the search space by keeping two indexes (low and high) that initially point to two end-points of the subarray. 
Then we loop until low is less than high and reduce the search space arr[low...high] at each iteration of the loop.
We compare the sum of the values present at index low and high with r and increment low if the sum is less than r 
and decrement high if the sum is more than r. Finally, if the sum is equal to r, we found the desired pair.
The quadruplet will then consist of the initial pair we found in the first step and the complementing pair we 
found in the subarray.

Time Complexity: we have three nested loops whose combined time complexity is O(N^3),
where N is the size of arr. We also using sorting in the beginning and that’s additional O(N⋅log(N)). 
The total time complexity is still O(N^3) because O(N⋅log(N)) gets thrown away since in the asymptotic 
calculation it’s not material.

Space Complexity: O(1) as we used only a constant amount of space throughout the algorithm.
"""
def find_array_quadruplet(arr,s):
    n = len(arr)
    # if there are fewer than 4 items in arr, by
    # definition no quadruplet exists whose sum is s
    if (n < 4):
        return []
    # sort arr in an ascending order
    arr.sort()
    for i in range(0, n - 4):
        for j in range (i + 1, n - 3):
            # r stores the complementing sum
            r = s - (arr[i] + arr[j])

            # check for sum r in subarray arr[j+1…n-1]
            low = j + 1
            high = n - 1
            
            while (low < high):
                if (arr[low] + arr[high] < r):
                    low += 1
                elif (arr[low] + arr[high] > r):
                    high -= 1
                # quadruplet with given sum found
                else:
                    return [arr[i], arr[j], arr[low], arr[high]]
    return []

arr,s = [], 20
print(find_array_quadruplet(arr,s)) #[]
arr,s = [4,4,4], 16
print(find_array_quadruplet(arr,s)) #[]
arr,s = [2, 7, 4, 0, 9, 5, 1, 3], 20
print(find_array_quadruplet(arr,s)) #[0, 4, 7, 9]
arr, s = [1,2,3,4,5,9,19,12,12,19], 40
print(find_array_quadruplet(arr,s)) #[4,5,12,19]

[]
[]
[0, 4, 7, 9]
[4, 5, 12, 19]


In [ ]:
"""

Example (more examples can be found here):

Write a readable an efficient code, explain how it
is built and why you chose to build it that way.

Sudoku Solver
Get your peer to build a
skeleton of what the program should do at a high level, using 
pseudocode and skipping over helper functions.

If your peer is stuck, ask what a brute force solution might look like.

What the program should do at a high level is this: choose 
an empty cell and place values inside. If some placed value 
makes solve(board) true, then the answer is true - otherwise,
the answer is false.

If your peer has not considered it or is choosing the “empty cell” 
naively, ask them what choice of empty cell would be best. 
If still stuck, clarify that we want to choose an empty cell 
that will have us do the least work later.

The best choice of an empty cell is the choice with the least
possibilities, because in the worst case we will check 
sudokuSolve(board) the least times. Encourage your peer to
write a helper function getCandidates that gets the 
possibilities of what values some cell board[r][c] could be.

If your peer is still stuck on the skeleton or getting
started after a considerable time, here is a good start:

function getCandidates(board, row, col):
  # What values can be placed in empty cell board[row][col] ?
 
function sudokuSolve(board):
  # For each empty cell (row, col):
  #   If (row, col) has fewer candidate values that can be placed
  #   in board[row][col]than we've seen, remember it
 
  # If there's no empty cell:
  #   return true
 
  # For candidate values v of our remembered row, col:
  #   board[row][col] = v
  #   if solve(board): 
  #     return true
    
  # return false


"""


In [16]:
"""
Getting a Different Number
Given an array arr of unique nonnegative integers, implement a function
getDifferentNumber that finds the smallest nonnegative integer that
is NOT in the array.

Even if your programming language of choice doesn’t have that 
restriction (like Python), assume that the maximum value an integer
can have is MAX_INT = 2^31-1. So, for instance, the operation MAX_INT + 1 
would be undefined in our case.

Your algorithm should be efficient, both from a time and a space complexity
perspectives.

Solve first for the case when you’re NOT allowed to modify the input arr. 
If successful and still have time, see if you can come up with an algorithm
with an improved space complexity when modifying arr is allowed. 
Do so without trading off the time complexity.

Example:

input:  arr = [0, 1, 2, 3]
output: 4 

"""
def get_different_number(arr): # O(N), O(N)
    """ Applicable if we're NOT allowed to modify the array. """
    
    length = len(arr)
    reference_arr = set()
    for x in arr:
        reference_arr.add(x)

    missing = None

    for i in range(0, length):
        if i not in reference_arr:
            missing = i
            return missing

    return length


def get_different_number_inPlace(arr): # O(N), O(1)
    """ Only applicable if we're allowed to modify the array.
    
    Time Complexity: 
        At first glance, one might think that due to the two 
        nested loops (a while loop inside a for loop) that we use to sort
        the array, the time complexity is O(N^2). However, this is incorrect.
        The actual time complexity of the two nested loops is linear. 
        The reason is that every number is at most moved once. 
        For those already in their target indices, the while loop will end
        immediately since the condition arr[temp] != temp isn’t met. 
        In the second part of the code we have another loop whose time complexity
        is linear. The total time complexity is therefore O(N).

    Space Complexity: 
        We use only constant space. Hence the space complexity is O(1).
    """
    length = len(arr)
    temp = 0
    
    # Put each number in its corresponding index, kicking out
    # the original number, until the target index is out of range.
    for i in range(0, length-1):
        temp = arr[i]
        while (temp < length and arr[temp] != temp):
            temp, arr[temp] = arr[temp], temp
    
    for i in range(0, length-1):
        if (arr[i] != i):
            return i # i isn’t in arr, hence we can return it
    
    # we got here since every number from 0 to n-1 is in arr.
    # By definition then, n isn’t in arr. Otherwise, the size of arr
    # would have been n+1 and not n.
    return length
    
print("Expected 1: ", get_different_number([0]))               # Expected 1
print("Expected 3: ", get_different_number([0,1,2]))           # Expected 3
print("Expected 4: ", get_different_number([1,3,0,2]))         # Expected 4
print("Expected 0: ", get_different_number([100000]))          # Expected 0
print("Expected 2: ", get_different_number([1,0,3,4,5]))       # Expected 2
print("Expected 1: ", get_different_number([0,100000]))        # Expected 1
print("Expected 1: ", get_different_number([0,99999,100000]))  # Expected 1
print("Expected 7: ", get_different_number([0,5,4,1,3,6,2]))   # Expected 7

# print("Expected 1: ", get_different_number_inPlace([0]))               # Expected 1
# print("Expected 3: ", get_different_number_inPlace([0,1,2]))           # Expected 3
# print("Expected 4: ", get_different_number_inPlace([1,3,0,2]))         # Expected 4
# print("Expected 0: ", get_different_number_inPlace([100000]))          # Expected 0
# print("Expected 2: ", get_different_number_inPlace([1,0,3,4,5]))       # Expected 2
# print("Expected 1: ", get_different_number_inPlace([0,100000]))        # Expected 1
# print("Expected 1: ", get_different_number_inPlace([0,99999,100000]))  # Expected 1
# print("Expected 7: ", get_different_number_inPlace([0,5,4,1,3,6,2]))   # Expected 7

Expected 1:  1
Expected 3:  3
Expected 4:  4
Expected 0:  0
Expected 2:  2
Expected 1:  1
Expected 1:  1
Expected 7:  7


In [14]:
"""
Sudoku Solver
Write the function sudokuSolve that checks whether a given sudoku board 
(i.e. sudoku puzzle) is solvable. If so, the function will returns true.
Otherwise (i.e. there is no valid solution to the given sudoku board), 
returns false.

In sudoku, the objective is to fill a 9x9 board with digits so that each 
column, each row, and each of the nine 3x3 sub-boards that compose the 
board contains all of the digits from 1 to 9. The board setter provides 
a partially completed board, which for a well-posed board has a unique 
solution. As explained above, for this problem, it suffices to calculate 
whether a given sudoku board has a solution. No need to return the actual
numbers that make up a solution.

A sudoku board is represented as a two-dimensional 9x9 array of the 
characters ‘1’,‘2’,…,‘9’ and the '.' character, which represents a 
blank space. The function should fill the blank spaces with characters
such that the following rules apply:

In every row of the array, all characters ‘1’,‘2’,…,‘9’ appear
exactly once.
In every column of the array, all characters ‘1’,‘2’,…,‘9’ appear
exactly once.
In every 3x3 sub-board that is illustrated below, all characters
‘1’,‘2’,…,‘9’ appear exactly once.
A solved sudoku is a board with no blank spaces, i.e. all blank 
spaces are filled with characters that abide to the constraints
above. If the function succeeds in solving the sudoku board, 
it’ll return true (false, otherwise).

The most straightforward way to build a sudoku solver is a recursive
backtracking algorithm. In such an algorithm, we change one cell of
the board (possibly multiple times) and call our function again to 
ask whether that board can be solved.

"""

import math

def sudoku_solve(board):
    """ The main idea for the solution is using a recursive algorithm 
        that iterates through the empty cell on the board, and on the 
        possible candidate numbers for each empty cell, and tries to
        fill them. """
    # For each empty cell, consider 'newCandidates', the
    # set of possible candidate values that can
    # be placed into that cell.
    row = -1 
    col = -1
    candidates = None 
    for r in range(0, 9):
        for c in range(0, 9):
            if (board[r][c] == '.'):
                newCandidates = getCandidates(board, r, c)
                # Then, we want to keep the smallest
                # sized 'newCandidates', plus remember the
                # position where it was found
                if (candidates == None) or (len(newCandidates) < len(candidates)):
                    candidates = newCandidates
                    row = r 
                    col = c
    # If we have not found any empty cell, then
    # the whole board is filled already
    if (candidates == None):
        return True
    
    # For each possible value that can be placed
    # in position (row, col), let's
    # place that value and then recursively query
    # whether the board can be solved.  If it can,
    # we are done. 
    for val in candidates:
        board[row][col] = val
        if (sudoku_solve(board)) == True:
            return True
        else:
            # The tried value val didn't work so restore  
            # the (row, col) cell back to '.'
            board[row][col] = '.'
    
    # Otherwise, there is no value that can be placed
    # into position (row, col) to make the
    # board solved
    return False

def getCandidates(board, row, col):
    """ A helper function that returns a set of all valid
        candidates for a given cell in the board """
    # For some empty cell board[row][col], what possible
    # characters can be placed into this cell
    # that aren't already placed in the same row,
    # column, and sub-board?
    # At the beginning, we don't have any candidates
    candidates = []
    
    # For each character add it to the candidate list
    # only if there's no collision, i.e. that character 
    # doesn't already exist in the same row, column 
    # and sub-board. Notice the top-left corner of (row, col)'s 
    # sub-board is (row - row%3, col - col%3).
    ref = ["1", "2", "3", "4", "5", "6", "7", "8", "9"]
    for chr in ref:
        collision = False
        for i in range(0, 9):
            top_left_x = int((row - row % 3) + math.floor(i / 3))
            top_left_y = int((col - col % 3) + i % 3)
            if (board[row][i] == chr or 
                board[i][col] == chr or
                board[top_left_x][top_left_y] == chr):
                    collision = True
                    break
                  
        if (collision == False):
            candidates.append(chr)
    return candidates

# Test Case 1
# Expected: True
input1 = [[".",".",".","7",".",".","3",".","1"],
          ["3",".",".","9",".",".",".",".","."],
          [".","4",".","3","1",".","2",".","."],
          [".","6",".","4",".",".","5",".","."],
          [".",".",".",".",".",".",".",".","."],
          [".",".","1",".",".","8",".","4","."],
          [".",".","6",".","2","1",".","5","."],
          [".",".",".",".",".","9",".",".","8"],
          ["8",".","5",".",".","4",".",".","."]]

print("Expected: True. Actual: ", sudoku_solve(input1))

# Test Case 2
# Expected: False 
input2= [[".","8","9",".","4",".","6",".","5"],
        [".","7",".",".",".","8",".","4","1"],
        ["5","6",".","9",".",".",".",".","8"],
        [".",".",".","7",".","5",".","9","."],
        [".","9",".","4",".","1",".","5","."],
        [".","3",".","9",".","6",".","1","."],
        ["8",".",".",".",".",".",".",".","7"],
        [".","2",".","8",".",".",".","6","."],
        [".",".","6",".","7",".",".","8","."]]

print("Expected: False. Actual: ", sudoku_solve(input2))

# Test Case 3
# Expected: False
input3 = [[".","2","3","4","5","6","7","8","9"],
          ["1",".",".",".",".",".",".",".","."],
          [".",".",".",".",".",".",".",".","."],
          [".",".",".",".",".",".",".",".","."],
          [".",".",".",".",".",".",".",".","."],
          [".",".",".",".",".",".",".",".","."],
          [".",".",".",".",".",".",".",".","."],
          [".",".",".",".",".",".",".",".","."],
          [".",".",".",".",".",".",".",".","."]]

print("Expected: False. Actual: ", sudoku_solve(input3))

# Test Case 4
# Expected: True
input4 = [[".",".","5",".",".","2",".",".","."],
          [".",".","9",".","4","7",".","2","."],
          [".",".","8",".","5","6",".",".","1"],
          [".",".",".",".",".","8","3","4","."],
          [".",".",".",".",".",".",".",".","6"],
          [".",".",".",".","3",".","1","8","."],
          [".","2",".",".",".",".",".",".","."],
          [".","9",".",".","8",".","6","7","."],
          ["3",".","6","5","7",".",".",".","."]]

print("Expected: True. Actual: ", sudoku_solve(input4))

# Test Case 5
# Expected: True
input5 = [[".",".","3","8",".",".","4",".","."],
          [".",".",".",".","1",".",".","7","."],
          [".","6",".",".",".","5",".",".","9"],
          [".",".",".","9",".",".","6",".","."],
          [".","2",".",".",".",".",".","1","."],
          [".",".","4",".",".","3",".",".","2"],
          [".",".","2",".",".",".","8",".","."],
          [".","1",".",".",".",".",".","5","."],
          ["9",".",".",".",".","7",".",".","3"]]

print("Expected: True. Actual: ", sudoku_solve(input5))

# Test Case 6
# Expected: True
input6 = [[".",".",".",".",".",".",".",".","."],
          [".",".",".",".",".",".",".",".","."],
          [".",".",".",".",".",".",".",".","."],
          [".",".",".",".",".",".",".",".","."],
          [".",".",".",".",".",".",".",".","."],
          [".",".",".",".",".",".",".",".","."],
          [".",".",".",".",".",".",".",".","."],
          [".",".",".",".",".",".",".",".","."],
          [".",".",".",".",".",".",".",".","."]]

print("Expected: True. Actual: ", sudoku_solve(input6))

Expected: True. Actual:  True
Expected: False. Actual:  False
Expected: False. Actual:  False
Expected: True. Actual:  True
Expected: True. Actual:  True
Expected: True. Actual:  True
